## Learn about class and pytorch

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Build data loader

In [2]:
import pyarrow.feather as feather
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd



In [3]:
def build_dataset(batchSizeTrain, batchsizeValid):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    # normalize input data
    fluxData_df_norm = (fluxData_df - fluxData_df.mean(axis=0)) / fluxData_df.std(axis=0)
    zernikeData_df = feather.read_feather('data/zernikeData.feather')
    X_train, X_val, y_train, y_val = train_test_split(fluxData_df_norm, zernikeData_df, test_size=0.2, random_state=42)


    train_target = torch.tensor(y_train.values.astype(np.float32))
    trainInput = torch.tensor(X_train.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(trainInput, train_target) 
    loaderTrain = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchSizeTrain, shuffle = True)


    valid_target = torch.tensor(y_val.values.astype(np.float32))
    validInput = torch.tensor(X_val.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(validInput, valid_target) 
    loaderValid = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batchsizeValid, shuffle = True)
    return loaderTrain, loaderValid



# Build data loader

## Build AO network as class

In [4]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(19,2000)
        self.linear2 = nn.Linear(2000,1050)
        self.linear3 = nn.Linear(1050,100)
        self.out = nn.Linear(100,9)
        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU()
    })

    def forward(self, x, act):
        x = self.linear1(x)
        x = self.activations[act](x)
        x = self.linear2(x)
        x = self.activations[act](x)
        x = self.linear3(x)
        x = self.activations[act](x)
        x = self.out(x)
        return(x)

ClassNetwork = Net()
optimizer = optim.SGD(ClassNetwork.parameters(), lr=0.001, momentum=0.9)
loaderTrain, loaderValid =build_dataset(1024,1024)



In [5]:
import wandb
wandb.init(project="test")



wandb: Currently logged in as: uwe-sterr (use `wandb login --relogin` to force relogin)


In [6]:
def train():
    ClassNetwork = Net()
    wandb.watch(ClassNetwork, log_freq=100)
    optimizer = optim.SGD(ClassNetwork.parameters(), lr=0.001, momentum=0.9)
    loaderTrain, loaderValid =build_dataset(1024,1024)
    epochs = 200
    for epoch in range(epochs):
        for i, data in enumerate(loaderValid,0):
            input, labels = data
            optimizer.zero_grad()
            outputs = ClassNetwork(input, "relu")
            loss = nn.MSELoss()
            loss =loss(ClassNetwork(input, "relu"), labels)
            RmsLossValid=torch.sqrt(loss)
    # ⬅ Backward pass + weight update
            loss.backward()
            optimizer.step()
        print("epoch: ", epoch, "loss: ", "%.4f" % loss.item() ,"RmsLossValid: " , "%.4f" % RmsLossValid.item())  
        #wandb.log({"loss": loss.item(), "RmsLossValid": RmsLossValid.item()})
        wandb.log({"loss": 5})

In [7]:
train()

epoch:  0 loss:  0.0390 RmsLossValid:  0.1976


Exception: The wandb backend process has shutdown

Error in callback <function _WandbInit._pause_backend at 0x7f2ee5571dc0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [ ]:
loss.item()